In [48]:
import requests
import pandas as pd 
from sqlalchemy import create_engine , text
from dotenv import load_dotenv
import os

In [49]:
load_dotenv()

True

In [50]:
api_data = requests.get("https://fantasy.premierleague.com/api/bootstrap-static/")
json_data = api_data.json()

In [51]:
df = pd.DataFrame(json_data["teams"])

In [52]:
df.drop(
    columns=["code", "id", "team_division", "unavailable", "pulse_id"],
    inplace=True
)

In [53]:
df.drop(
    columns=["form"],
    inplace=True
)

In [54]:
df.rename(
    columns={
        "name" : "team_name",
        "short_name" : "team_short_name",
        "strength" : "team_strength"
    },
    inplace=True
)

In [55]:
postgres_URI = os.getenv("POSTGRES_URI") or ""
engine = create_engine(postgres_URI)

In [56]:
with engine.begin() as conn:
    conn.execute(
        text(
            '''
            create table if not exists stg_teams(
                team_id int generated always as identity primary key not null,
                team_name varchar(100) not null,
                draw int ,
                loss int,
                played int,
                points int,
                position int,
                win int,
                team_short_name varchar(50) not null,
                team_strength int, 
                strength_overall_home int ,
                strength_overall_away int, 
                strength_attack_home int ,
                strength_attack_away int,
                strength_defence_home int,
                strength_defence_away int
            );
            '''
        )
    )
    conn.execute(
        text(
            "truncate stg_teams restart identity"
        )
    )

In [57]:
df.to_sql(
    con=engine,
    name="stg_teams",
    if_exists="append",
    index=False
)

20

In [58]:
with engine.begin() as con:
    con.execute(
        text(
            '''
            MERGE INTO teams as target 
            USING stg_teams as source
            ON target.team_id = source.team_id
            
            WHEN MATCHED THEN 
                UPDATE SET 
                    team_name = source.team_name,
                    draw = source.draw,
                    loss = source.loss,
                    played = source.played ,
                    points = source.points,
                    position = source.position,
                    win = source.win ,
                    team_short_name = source.team_short_name, 
                    team_strength = source.team_strength,
                    strength_overall_home =  source.strength_overall_home,
                    strength_overall_away = source.strength_overall_away, 
                    strength_attack_home =  source.strength_attack_home,
                    strength_attack_away = source.strength_attack_away,
                    strength_defence_home = source.strength_defence_home,
                    strength_defence_away = source.strength_defence_away
                    
            WHEN NOT MATCHED THEN 
                INSERT (team_name, draw, loss, played, points, position, win, team_short_name, team_strength, strength_overall_home, strength_overall_away, strength_attack_home, strength_attack_away, strength_defence_home, strength_defence_away)
                VALUES (source.team_name, source.draw, source.loss, source.played, source.points, source.position, source.win, source.team_short_name, source.team_strength, source.strength_overall_home, source.strength_overall_away, source.strength_attack_home, source.strength_attack_away, source.strength_defence_home, source.strength_defence_away)
            '''
        )
    )